In [1]:
%pip install --upgrade baolib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 10.4 MB/s  0:00:00eta 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import tempfile
from pathlib import Path

from baolib.bao import (
    Access,
    AccessChange,
    Bao,
    DB,
    Groups,
    Message,
    newPrivateID,
    publicID,
    set_bao_log_level,
)

set_bao_log_level('info')


/Users/ea/Documents/Lab/bao/.venv/lib/python3.14/site-packages/baolib/baod.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


INFO  [06:07:12] (+30ms) core.syncTime[time.go:34] - clock offset 14.230081ms from time.google.com 


# Introduction
Bao is a secure storage library for sharing encrypted data. This notebook shows the Python binding (`baolib`) API.


## Identity
Create identities with `newPrivateID()` and derive the public ID with `publicID()`.


In [5]:
owner_private = newPrivateID()
owner_public = publicID(owner_private)

print('Owner public ID:', owner_public)


Owner public ID: AuZdksORpdkgE0IPZ25M9dEiLR4dThVpg78yFipiHbqCS-QOyaOrrnhfKrp43gLuhQb8cQWlLiy9vpezGMXyzkM=


## Local database
Use a local SQLite database to track metadata. Here we start with a clean DB in a temp directory.


In [6]:
temp_root = Path(tempfile.gettempdir()) / 'bao_python_sample'
temp_root.mkdir(parents=True, exist_ok=True)

db_path = temp_root / 'bao.db'
if db_path.exists():
    db_path.unlink()

db = DB.open('sqlite3', str(db_path))
print('Database at', db_path)


Database at /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao.db


INFO  [06:07:17] (+1ms) sqlx.Open[db.go:63] - successfully opened SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao.db with driver sqlite3


## Create a Bao vault
A Bao vault combines the database, an identity, and a storage URL. The storage can be local (`file://`), S3, SFTP, etc.


In [7]:
store_url = f'file://{temp_root}/vault'
vault = Bao.create(db, owner_private, store_url)
vault


INFO  [06:07:19] (+11ms) Successfully wiped data from store file:///var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault
INFO  [06:07:19] (+12ms) bao.(*Bao).convertToChanges[access.go:154] - successfully created 2 changes for group admins
INFO  [06:07:20] (+1020ms) bao.(*ChangeAccess).Apply[changes.go:404] - my access for group admins changed to A
INFO  [06:07:20] (+1022ms) bao.Create[create.go:80] - Successfully created Bao instance for url file:///var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault: accesses:
  admins:
    AuZdksORpdkgE0IPZ25M9dEiLR4dThVpg78yFipiHbqCS-QOyaOrrnhfKrp43gLuhQb8cQWlLiy9vpezGMXyzkM=: 3
author: AuZdksORpdkgE0IPZ25M9dEiLR4dThVpg78yFipiHbqCS-QOyaOrrnhfKrp43gLuhQb8cQWlLiy9vpezGMXyzkM=
config:
  retention: 0s
  maxstorage: 0
  segmentinterval: 0s
  synctimeout: 0s
  syncperiod: 0s
  filessyncperiod: 0s
  cleanupperiod: 0s
  blockchainsyncperiod: 0s
  iothrottle: 0
  chainrepair: 0
id: bzPZoC4nLtLBCaOvZr8BgXOfm7rfjRFl-2NVLJ

file:///var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault

## Access control
Grant permissions to groups. Built-in groups include `admins`, `users`, and `public`. Permissions are bitmasks (`Access.read`, `Access.write`, `Access.admin`).


In [8]:
vault.sync_access([
    AccessChange(Groups.users, Access.read_write, owner_public),
])

collaborator_private = newPrivateID()
collaborator_public = publicID(collaborator_private)
vault.sync_access([
    AccessChange(Groups.users, Access.read, collaborator_public),
])

vault.get_access(Groups.users)


INFO  [06:07:21] (+1ms) bao.(*Bao).convertToChanges[access.go:154] - successfully created 2 changes for group users
INFO  [06:07:22] (+1011ms) bao.(*ChangeAccess).Apply[changes.go:404] - my access for group users changed to A
INFO  [06:07:22] (+0ms) bao.(*Bao).convertToChanges[access.go:154] - successfully created 2 changes for group users


{'Ag0HugBL0KDJNomEpkpdEC3Rc51kpOgdZMwqO0xjkKOhCMgvRwuP2i2MomJGDuV-6GZYSgLUM9TwbfRxHLqONGk=': 1,
 'AuZdksORpdkgE0IPZ25M9dEiLR4dThVpg78yFipiHbqCS-QOyaOrrnhfKrp43gLuhQb8cQWlLiy9vpezGMXyzkM=': 3}

## Open an existing vault
Anyone with the vault URL, their private ID, and the creator's public ID can reopen the vault.


In [9]:
vault = Bao.open(db, owner_private, store_url, owner_public)
vault


INFO  [06:07:24] (+0ms) bao.Open[open.go:79] - successfully opened Bao instance with store URL file:///var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault


file:///var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault

## Files and data
Write and read files using group permissions.


In [10]:
data_path = temp_root / 'hello.txt'
data_path.write_text('Hello from Bao!')

vault.write('docs/hello.txt', Groups.users, src=str(data_path))
vault.read_dir('docs', None, 0, 10)


[{'id': 1,
  'name': 'hello.txt',
  'group': 'users',
  'size': 15,
  'allocatedSize': 215,
  'modTime': '2025-12-19T06:07:26.202+01:00',
  'isDir': False,
  'flags': 0,
  'local': '/var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.txt',
  'keyId': 392543408559550464,
  'storageDir': 'data/users/20251219000000',
  'storageName': '2ayu28m8',
  'authorId': 'AuZdksORpdkgE0IPZ25M9dEiLR4dThVpg78yFipiHbqCS-QOyaOrrnhfKrp43gLuhQb8cQWlLiy9vpezGMXyzkM='}]

In [11]:
out_path = temp_root / 'hello.out'
vault.read('docs/hello.txt', str(out_path))
out_path.read_text()


INFO  [06:07:26] (+3ms) bao.(*Bao).Read[read.go:142] - successfully read file docs/hello.txt to /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.out in 3.349833ms


'Hello from Bao!'

## Messaging
Send and receive `Message` objects inside a vault directory.


In [12]:
vault.send('mailbox', Groups.users, Message('Greetings', 'Hello team!'))
vault.receive('mailbox', 0, 0)


[Message(subject='Greetings', body='Hello team!', attachments=[], fileInfo={'id': 2, 'name': '572982df6dcf000', 'group': 'users', 'size': 0, 'allocatedSize': 456, 'modTime': '2025-12-19T06:07:29.386+01:00', 'isDir': False, 'flags': 0, 'attrs': 'eyJzdWJqZWN0IjoiR3JlZXRpbmdzIiwiYm9keSI6IkhlbGxvIHRlYW0hIiwiYXR0YWNobWVudHMiOltdLCJmaWxlSW5mbyI6eyJpZCI6MCwibmFtZSI6IiIsImdyb3VwIjoiIiwic2l6ZSI6MCwiYWxsb2NhdGVkU2l6ZSI6MCwibW9kVGltZSI6IjAwMDEtMDEtMDFUMDA6MDA6MDBaIiwiaXNEaXIiOmZhbHNlLCJmbGFncyI6MCwia2V5SWQiOjAsInN0b3JhZ2VEaXIiOiIiLCJzdG9yYWdlTmFtZSI6IiIsImF1dGhvcklkIjoiIn19', 'keyId': 392543408559550464, 'storageDir': 'data/users/20251219000000', 'storageName': '2az9lan4', 'authorId': 'AuZdksORpdkgE0IPZ25M9dEiLR4dThVpg78yFipiHbqCS-QOyaOrrnhfKrp43gLuhQb8cQWlLiy9vpezGMXyzkM='})]

## SQL layer
Attach a SQL layer that replicates statements through the vault.


In [14]:
ddl = '''
-- INIT 1.0
CREATE TABLE IF NOT EXISTS notes (id INTEGER PRIMARY KEY, body TEXT);

-- INSERT_NOTE 1.0
INSERT INTO notes (id, body) VALUES (:id, :body);

-- SELECT_ALL_NOTES 1.0
SELECT * FROM notes;
'''

db_path = temp_root / 'data.db'
if db_path.exists():
    db_path.unlink()
    
data_db = DB.open('sqlite3', str(db_path), ddl=ddl)
sql = vault.baoql(Groups.sql, data_db)
sql.exec('INSERT_NOTE', {'id': 1, 'body': 'hello'})
sql.exec('INSERT_NOTE', {'id': 2, 'body': 'bao'})
sql.fetch('SELECT_ALL_NOTES', {})

INFO  [06:08:47] (+2ms) sqlx.Open[db.go:63] - successfully opened SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/data.db with driver sqlite3
INFO  [06:08:47] (+0ms) sqlx.(*DB).Close[db.go:74] - successfully closed SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/data.db


[[1, 'hello'], [2, 'bao']]

INFO  [06:37:12] (+1704614ms) core.syncTime[time.go:34] - clock offset 24.172285ms from time.google.com 
INFO  [07:07:12] (+3504493ms) core.syncTime[time.go:34] - clock offset 27.810362ms from time.google.com 
INFO  [07:58:32] (+5304530ms) core.syncTime[time.go:34] - clock offset 114.71669ms from time.google.com 
INFO  [08:56:48] (+7104496ms) core.syncTime[time.go:34] - clock offset 55.31572ms from time.google.com 
